In [ ]:
import csv
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging
import re

# Set up basic configuration for logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# OpenAI client
client = OpenAI()

# List of cuisines
cuisine_types = [
    "Cantonese", "Sichuan", "Shandong", "Jiangsu", "Zhejiang",
    "Hunan", "Anhui", "Fujian", "Italian", "Indian", "French",
    "Mexican", "Japanese", "Thai", "Spanish", "Greek",
    "Turkish", "Korean"
]

# Number of reviews per cuisine per request
reviews_per_cuisine = 10

def fetch_reviews(cuisine):
    try:
        logging.info(f"Fetching 10 reviews for {cuisine} cuisine")
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": f"Now you are a customer who has been to many restaurants specializing in {cuisine} cuisine."},
                {"role": "user", "content": f"Write 10 restaurant reviews about {cuisine} cuisine in simplified Chinese. The number of Chinese characters should be 600. The format of generated reviews should be pure text without any quotes and there is no numbered bullet point. Each review should be in its own line. Do exactly what I say."}
            ]
        )
        reviews = completion.choices[0].message.content.split('\n\n')
        logging.info(f"Successfully fetched reviews for {cuisine} cuisine")
        return reviews
    except Exception as e:
        logging.error(f"Error fetching reviews for {cuisine} cuisine: {e}")
        return []

# Total reviews needed
total_reviews = 15000

# Using ThreadPoolExecutor to fetch reviews for each cuisine in parallel
review_data = []
# Calculate the number of cuisines processed per round (the minimum of threads and number of cuisines)
cuisines_per_round = min(len(cuisine_types), 16)

# Calculate reviews fetched per round
reviews_per_round = cuisines_per_round * reviews_per_cuisine

# Calculate the total number of rounds needed, rounding up as you can't have a fraction of a round
rounds_needed = -(-total_reviews // reviews_per_round)

print(f"total {rounds_needed} needed to get reviews")

with ThreadPoolExecutor(max_workers=16) as executor:
    for _ in range(rounds_needed):
        future_to_cuisine = {executor.submit(fetch_reviews, cuisine): cuisine for cuisine in cuisine_types}
        for future in as_completed(future_to_cuisine):
            cuisine = future_to_cuisine[future]
            try:
                data = future.result()
                cleaned_messages = [re.sub(r'^\d+\.\s+', '', message) for message in data]
                # Writing to a CSV file
                with open('chinese_reviews_new.csv', 'a', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    for cleaned_msg in cleaned_messages:
                        writer.writerow([cleaned_msg])
                    logging.info(f"reviews successfully written to CSV")
                # review_data.extend(data)
            except Exception as e:
                logging.error(f"Error processing results for {cuisine} cuisine: {e}")


2023-11-29 01:44:33,141 - INFO - Fetching 10 reviews for Cantonese cuisine
2023-11-29 01:44:33,145 - INFO - Fetching 10 reviews for Sichuan cuisine
2023-11-29 01:44:33,145 - INFO - Fetching 10 reviews for Shandong cuisine
2023-11-29 01:44:33,145 - INFO - Fetching 10 reviews for Jiangsu cuisine
2023-11-29 01:44:33,154 - INFO - Fetching 10 reviews for Zhejiang cuisine
2023-11-29 01:44:33,155 - INFO - Fetching 10 reviews for Hunan cuisine
2023-11-29 01:44:33,155 - INFO - Fetching 10 reviews for Anhui cuisine
2023-11-29 01:44:33,159 - INFO - Fetching 10 reviews for Fujian cuisine
2023-11-29 01:44:33,160 - INFO - Fetching 10 reviews for Italian cuisine
2023-11-29 01:44:33,160 - INFO - Fetching 10 reviews for Indian cuisine
2023-11-29 01:44:33,169 - INFO - Fetching 10 reviews for French cuisine
2023-11-29 01:44:33,185 - INFO - Fetching 10 reviews for Mexican cuisine
2023-11-29 01:44:33,185 - INFO - Fetching 10 reviews for Japanese cuisine
2023-11-29 01:44:33,188 - INFO - Fetching 10 reviews 

total 94 needed to get reviews


2023-11-29 01:45:06,536 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-11-29 01:45:06,573 - INFO - Successfully fetched reviews for Sichuan cuisine
2023-11-29 01:45:06,574 - INFO - Fetching 10 reviews for Turkish cuisine
2023-11-29 01:45:06,575 - INFO - reviews successfully written to CSV
2023-11-29 01:45:10,819 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-11-29 01:45:10,828 - INFO - Successfully fetched reviews for Anhui cuisine
2023-11-29 01:45:10,830 - INFO - Fetching 10 reviews for Korean cuisine
2023-11-29 01:45:10,831 - INFO - reviews successfully written to CSV
2023-11-29 01:45:19,269 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-11-29 01:45:19,276 - INFO - Successfully fetched reviews for Fujian cuisine
2023-11-29 01:45:19,278 - INFO - reviews successfully written to CSV
2023-11-29 01:45:22,491 - INFO - HTTP Request: POST https://api.op

In [4]:
from openai import OpenAI

client = OpenAI()
completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": f"Now you are a customer who has been to many restaurants specializing in chinese cuisine."},
                {"role": "user", "content": f"Write 10 negative restaurant reviews in simplified Chinese. The number of Chinese characters should be 600. The format of generated reviews should be pure text without any quotes and there is no numbered bullet point. Each review should be in its own line. Do exactly what I say."}
            ]
        )
reviews = completion.choices[0].message.content.split('\n\n')

In [5]:
reviews

['1. 这家餐厅的服务态度非常差，服务员们不友善且懒散，完全没有顾客至上的精神。',
 '2. 餐厅的环境非常糟糕，桌子椅子脏兮兮的，厕所更是脏乱不堪，简直无法忍受。',
 '3. 虽然该餐厅被称为中餐馆，但他们的菜品却没有丝毫中国特色，味道平庸且缺乏创意。',
 '4. 这家餐厅的菜单上没有详细说明菜品的原材料和制作方式，这让我感到担忧，不知道食物的品质是否可靠。',
 '5. 我们点的菜品等了将近一个小时才上桌，而且各种菜都凉了，实在是让人生气和失望。',
 '6. 服务员对于客人提出的问题和要求反应迟钝，没有及时解答或者回应，让人感到不耐烦。',
 '7. 尽管价格不菲，但菜品的份量却非常小，无法满足食客的胃口，性价比太低。',
 '8. 我们点的菜品明明要求不辣，但实际上却是辛辣得让人难以承受，完全违背了我们的要求。',
 '9. 餐厅的卫生状况令人担忧，食材保存不当，有异味，让人怀疑菜品的新鲜度。',
 '10. 我们去的时候餐厅里人满为患，但服务员们明显应付不过来，导致整个用餐过程十分混乱和紊乱。']